---
# 1. Game of Life and Death - COVID19/Pandemic version

We START with a basic implimentation of the game of life, then move on to a more complicated COVID19 game of live <<<

This is an expansion of John Conway's game of life. Here we will add some complexity to the simulation and include a 'predator' (Green) that relies solely on the existance of feeder cells (Yellow) to survive. 

The feeder cells will follow these rules:
1. Any live cell with fewer than two live neighbours dies, as if by underpopulation.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by overpopulation.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

The predator cells will follow these rules:
1. Any live cell with no neighboring feeder cells dies.
* Any dead cell with at least one neighboring predator cell and more than 3 feeder cells becomes a new predator.

We will implement the following functions:
* `random_grid`
* `count_living_neighbors_feeder`
* `count_living_neighbors_predator`
* `get_neighbors`
* `update_grid`.

In [2]:
# You will need these imports
import numpy as np 

# Dont need these 3 until we run
import matplotlib.pyplot as plt  
from matplotlib import animation, rc 
from IPython.display import HTML

In [3]:
# Define some global variables - do not change these
### Think of these as similiar to diag up and left 
DEAD = 0
ALIVE_PREDATOR = 1
ALIVE_FEEDER = 2

In [4]:
def random_grid(dim, p_alive_feeder, p_alive_predator, seed): 
    """ Initialize a random grid of dim x dim random values
    
    Args:
        dim (int): dimensions for grid (will be square so x = y = dim)
        p_alive_feeder (float): probability of a live feeder cell on grid
        p_alive_predator (float): probability of a live predator cell on grid
        seed (int): seed for np.random.choice
    
    Returns:
        grid (numpy matrix): dim x dim numpy matrix populated with live and dead cells
        
    Example:
    >>> random_grid(4, .3, .3, 1) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    array([[0, 1, 2, 0], ...])
    """
    np.random.seed(seed) 
#     choices = (ALIVE_PREDATOR, ALIVE_FEEDER, DEAD)
    choices = (ALIVE_FEEDER, DEAD, ALIVE_PREDATOR)
#     probabilities = (p_alive_predator, p_alive_feeder, 1 - (p_alive_feeder + p_alive_predator))
    probabilities = (p_alive_feeder, 1 - (p_alive_feeder + p_alive_predator), p_alive_predator)
    grid = np.random.choice(choices, (dim, dim), p = probabilities)
    return grid 
# Order of variables being called in line 19 seemed to matter, it caused my doctest to fail. Sad#

In [5]:
def count_living_neighbors_feeder(row, column, grid):
    """ Count how many living feeder neighbors exist
    
    Args:
        row (int): row of cell                     #the index position of the row you are on, 
        column (int): column of cell               #the index position of the column you are on
        grid (np matrix): matrix of all cells
        
    Returns:
        living_count (int): number of living feeder neighbors
    
    Example:
    >>> count_living_neighbors_feeder(2,2, random_grid(4,.3,.3,1)) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    3
    """
    neighbors = get_neighbors(row, column, grid)
    return neighbors.count(ALIVE_FEEDER)

In [6]:
def count_living_neighbors_predator(row, column, grid):
    """ Count how many living predator neighbors exist
    
    Args:
        row (int): row of cell
        column (int): column of cell
        grid (np matrix): matrix of all cells
        
    Returns:
        living_count (int): number of living predator neighbors
    
    Example:
    >>> count_living_neighbors_predator(2,2, random_grid(4,.3,.3,1)) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    1
    """
    neighbors = get_neighbors(row, column, grid)
    
    return neighbors.count(ALIVE_PREDATOR)

In [7]:
def get_neighbors(row, column, grid):
    """ Get the neighboring cells
    excluding any that might be out of bounds
    
    Args:
        row (int): row of cell
        column (int): column of cell
        grid (np matrix): matrix of all cells
        
    Returns:
        neighbors (list): list of adjacent cells
        
    Example:
    >>> get_neighbors(2,2, random_grid(4,.3,.3,1)) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    [2, 2, 0, 0, 0, 1, 2, 0]
    """
    back_row = 0 if row == 0 else row - 1
    back_column = 0 if column == 0 else column - 1

    current_cell_row = 0 if row == 0 else 1
    current_cell_col = 0 if column == 0 else 1
    
    neighborhood = grid[back_row: row + 2, back_column: column + 2].copy()
    neighborhood[current_cell_row][current_cell_col] = 13  # This number may be any integer above with a value greater than the number of states (e.g. DEAD, ALIVE_FEEDER, etc)
    
    neighbors = []
    for neighbor in neighborhood.flatten().tolist():
        if neighbor != 13:
            neighbors.append(neighbor)
    return neighbors

In [8]:
def update_grid(frameNum, img, grid, dim): 
    """ Update grid of cells based on rules:
    
    Any live cell with fewer than two live neighbours dies, as if by underpopulation.
    Any live cell with two or three live neighbours lives on to the next generation.
    Any live cell with more than three live neighbours dies, as if by overpopulation.
    Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

    Args:
        frameNum (int): frame number for the simulation (this is passed by animation)
        img (image object): this is passed by animation
        grid (np array): this is passed by animation
        dim (int): dimensions of grid (this is passed by animation)
        
    This function copies a new Grid over the existing grid for the animation process
    
    """
    working_copy = np.copy(grid)
    for row in range(dim):
        for col in range(dim):
            n_predators = count_living_neighbors_predator(row,col,working_copy)
            n_feeders = count_living_neighbors_feeder(row,col,working_copy)
            
            current_cell = working_copy[row][col]
        
            if current_cell == DEAD:
                if n_feeders == 3:
                    grid[row][col] = ALIVE_FEEDER
                elif n_predators >= 1 and n_feeders > 3:
                    grid[row][col] = ALIVE_PREDATOR
                    
            elif current_cell == ALIVE_PREDATOR:
                if n_feeders == 0:
                    grid[row][col] = DEAD
                    
            elif current_cell == ALIVE_FEEDER:
                if not 2 <= n_feeders <= 3:
                    grid[row][col] = DEAD
#                 if n_feeders < 2:
#                     grid[row][col] = DEAD
# #                 elif 2 <= n_feeders <= 3:
# #                     working_copy[i][j] = ALIVE_FEEDER
#                 elif n_feeders > 3:
#                     grid[row][col] = DEAD
    img.set_data(grid)

In [9]:
# This function does not need to be changed!
def run_simulation(grid_size = 100, p_alive_feeder = .5, p_alive_predator = .05, updateInterval = 500, seed=42): 
    """ Function to run the full simulation
    Each frame is an iteration of the model which calls update_grid
    with the arguments listed in fargs
    
    img should be updated with img.set_data(newGrid) in each iteration of update_grid
    
    """
    # declare grid 
    grid = np.array([]) 
    grid = random_grid(grid_size, p_alive_feeder, p_alive_predator, seed=seed) 
  
    # set up animation 
    fig, ax = plt.subplots(figsize=(6,6)) ### The inherent color coding regime fails in very high and low probability edge cases, it seems. We need to define the colors for 0 ,1, and 2, rather than having them arbitrarily assiging purple to the most prevalent value) 
    
    img = ax.imshow(grid) 
    anim = animation.FuncAnimation(fig, update_grid, fargs=(img, grid, grid_size, ), 
                                  frames = 100, 
                                  interval=updateInterval) 
    
    #anim.save('gol.gif', writer='imagemagick', fps=60)
    return HTML(anim.to_html5_video())

In [15]:
%%capture
game_of_life = run_simulation(p_alive_feeder = .75, p_alive_predator = .05, updateInterval = 500, seed=420)

In [16]:
game_of_life